## Chaine de traitement en Python pour adapter le moteur de reconnaissance d'entités nommées de Spacy par entrainement. 

Il s'agit de :


1.   annoter automatiquement en noms de lieux (LOC) et de personnes (PERS) un extrait du texte "Bel Ami" de Maupassant en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy ;
2.   transférer le texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations créées automatiquement;
3.   Corriger manuellement ces annotations en vue de créer un jeu  de données d'entrainement ;
4.   Formatter et intégrer les annotations corrigées dans Spacy en tant que jeu de données d'entrainement ;
5.  Initialiser un modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement ;
6.  Initialiser l'entrainement avec les paramètres définis dans le fichier .cfg et entrainement par le biais du CLI de Spacy ;
7.  Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant.





In [32]:
!git clone https://github.com/cvbrandoe/coursTAL.git

Cloning into 'coursTAL'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 167 (delta 47), reused 75 (delta 39), pack-reused 58
Receiving objects: 100% (167/167), 23.29 MiB | 23.24 MiB/s, done.
Resolving deltas: 100% (68/68), done.


**Prérequis : Installation ou mise à jour de Spacy et téléchargement du modèle**

In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 41.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!python -m spacy info

2022-12-07 04:01:26.497202: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================== Info about spaCy ==============================

spaCy version    3.4.3                         
Location         /usr/local/lib/python3.8/dist-packages/spacy
Platform         Linux-5.10.133+-x86_64-with-glibc2.27
Python version   3.8.15                        
Pipelines        en_core_web_sm (3.4.1)        



In [2]:
!python -m spacy download fr_core_news_sm  

2022-12-07 04:01:37.149863: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 53.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


Plus de modèles Spacy entrainés sur : https://spacy.io/models 

**Chargement d'un extrait du premier chapiture de Bel Ami de Maupassant.**


**1. Annotation automatiquement en noms de lieux (LOC) et de personnes (PERS) du texte en entrée en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy**

In [3]:
import spacy
import json
import requests
import os

def get_class_id(label):
  """
  Translates the spaCy label id into the tagtog entity type id
  - label: spaCy label id
  """
  choices = {'PER': 'e_1', 'LOC': 'e_2', 'GPE': 'e_2'}
  return choices.get(label, None)

def get_entities(spans, pipeline):
  """
  Translates a tuple of named entity Span objects (https://spacy.io/api/span) into a 
  list of tagtog entities (https://docs.tagtog.net/anndoc.html#ann-json). Each entity is
  defined by the entity type ID (classId), the part name where the annotation is (part),
  the entity offsets and the confidence (annotation status, who created it and probabilty).
  - spans: the named entities in the spaCy doc
  - pipeline: trained pipeline name
  """
  default_prob = 1
  default_part_id = 's1v1'
  default_state = 'pre-added'
  tagtog_entities = []
  for span in spans:
    class_id = get_class_id(span.label_)
    if class_id is not None:
      tagtog_entities.append( {
        'classId': class_id,
        'part': default_part_id,
        'offsets':[{'start': span.start_char, 'text': span.text}],
        'confidence': {'state': default_state,'who': ['ml:' + pipeline],'prob': default_prob},
        'fields':{},
        # this is related to the kb_id (knowledge base ID) field from the Span spaCy object
        'normalizations': {}} )
  return tagtog_entities


In [8]:
def read_text(file_name):
  with open (file_name, "r", encoding="utf_8") as myfile:
    lines = list(line for line in (l.strip() for l in myfile) if line)
    return str(lines)

text = read_text("coursTAL/2022/belAmi_01-01.txt").replace("’","'")

pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(text)

In [9]:
for token in doc:
  print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.is_stop)

[ [ PUNCT PUNCT punct False
' ' PUNCT PUNCT det False
PREMIÈRE premier ADJ ADJ amod True
PARTIE partie NOUN NOUN ROOT False
. . PUNCT PUNCT punct False
' ' PUNCT PUNCT advmod False
, , PUNCT PUNCT punct False
' ' PUNCT PUNCT punct False
I i NOUN NOUN conj True
' ' PUNCT PUNCT punct False
, , PUNCT PUNCT punct False
' ' PUNCT PUNCT punct False
Quand quand SCONJ SCONJ mark True
la le DET DET det True
caissière caissier NOUN NOUN nsubj False
lui lui PRON PRON iobj True
eut avoir VERB VERB aux:tense False
rendu rendre VERB VERB advcl False
la le DET DET det True
monnaie monnaie NOUN NOUN obj False
de de ADP ADP case True
sa son DET DET det True
pièce pièce NOUN NOUN nmod False
de de ADP ADP case True
cent cent NUM NUM nummod True
sous sous ADP ADP nmod True
, , PUNCT PUNCT punct False
Georges Georges PROPN PROPN nsubj False
Duroy Duroy PROPN PROPN flat:name False
sortit sortir VERB VERB ROOT False
du de ADP ADP case True
restaurant restaurant NOUN NOUN obl:arg False
. . PUNCT PUNCT punct F

In [15]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

In [16]:
def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
            print("\n")             
show_ents(doc)

PREMIÈRE PARTIE - 2 - 17 - ORG - Companies, agencies, institutions, etc.


I - 22 - 23 - LOC - Non-GPE locations, mountain ranges, bodies of water


Quand la caissière - 27 - 45 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Georges Duroy - 97 - 110 - PER - Named person or family.


joli garçon - 366 - 377 - PER - Named person or family.


Lorsqu' - 721 - 728 - LOC - Non-GPE locations, mountain ranges, bodies of water


rue Notre-Dame-de-Lorette - 1386 - 1411 - LOC - Non-GPE locations, mountain ranges, bodies of water


Quoique habillé d'un complet - 1967 - 1995 - ORG - Companies, agencies, institutions, etc.


C'était une de ces soirées d'été où l'air manque - 2419 - 2467 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Paris - 2473 - 2478 - LOC - Non-GPE locations, mountain ranges, bodies of water


Georges Duroy - 2981 - 2994 - PER - Named person or family.


Champs-Élysées - 3112 - 3126 - MISC - Miscell

In [17]:
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True, options={'distance': 90})

**2. transfer du texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations**

Au préalable, il faut créer un compte utilisateur et un projet dans TagTog (https://www.tagtog.com/) et modifier les informations de connexion ci-dessous.

Lors de la configuration (settings) de votre projet sur TagTog, dans la rubrique entities, il faut déclarer les trois catégories d'entités pour l'exercice (dans cet ordre là) :  **Personnages**, **Lieux** et **Misc** 

In [18]:
# Set the credentials at tagtog and project name
MY_USERNAME = 'ACHANGER'
MY_PASSWORD = 'ACHANGER'
MY_PROJECT = 'ACHANGER'

# API authentication
tagtogAPIUrl = "https://www.tagtog.com/-api/documents/v1"
auth = requests.auth.HTTPBasicAuth(username=MY_USERNAME, password=MY_PASSWORD)

# Initialize ann.json (specification: https://docs.tagtog.net/anndoc.html#ann-json)
annjson = {}
# Set the document as not confirmed, an annotator will manually confirm whether the annotations are correct
annjson['anncomplete'] = False
annjson['metas'] = {}
annjson['relations'] = []                      
# Transform the spaCy entities into tagtog entities
annjson['entities'] = get_entities(doc.ents, pipeline)

print(text)
print(json.dumps(annjson))

# Parameters for the API call 
# see https://docs.tagtog.net/API_documents_v1.html#examples-import-pre-annotated-plain-text-file
params = {'owner': MY_USERNAME, 'project': MY_PROJECT, 'output': 'null', 'format': 'default-plus-annjson'}
# Pre-annotated document composed of the content and the annotations
files=[('BelAmi_extrait0101.txt', text), ('BelAmi_extrait0101.ann.json', json.dumps(annjson))]
# POST request to send the pre-annotated document
response = requests.post(tagtogAPIUrl, params=params, auth=auth, files=files)

print(response.text)

['PREMIÈRE PARTIE.', 'I', 'Quand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant.', 'Comme il portait beau, par nature et par pose d'ancien sous-officier, il cambra sa taille, frisa sa moustache d'un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s'étendent comme des coups d'épervier.', 'Les femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d'un chapeau toujours poussiéreux et vêtue d'une robe toujours de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.', 'Lorsqu'il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu'il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans dîners, au choi

**3.  Correction manuelle des annotations en vue de créer un jeu  de données d'entrainement**

Ensuite téléchargez le fichier json à partir de tagtog et renommez-le en "BelAmi_01_01_corrected.ann.json" et importez-le dans l'espace de fichiers, ce fichier est traité ci-dessous.

**4. Formattage et intégration des annotations corrigées dans Spacy en tant que jeu de données d'entrainement**

In [19]:
# importing the module
import json
  # reading the data from the file
with open('coursTAL/2022/BelAmi_01_01_corrected.ann.json') as f:
    data = f.read()  
print("Data type before reconstruction : ", type(data))
# reconstructing the data as a dictionary
js_corrected_training_data = json.loads(data)
print("Data type after reconstruction : ", type(js_corrected_training_data))
print(js_corrected_training_data)

Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'dict'>
{'anncomplete': False, 'metas': {}, 'relations': [], 'entities': [{'classId': 'e_1', 'part': 's1v1', 'offsets': [{'start': 95, 'text': 'Georges Duroy'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_2', 'part': 's1v1', 'offsets': [{'start': 1378, 'text': 'rue Notre-Dame-de-Lorette'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_2', 'part': 's1v1', 'offsets': [{'start': 2459, 'text': 'Paris'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['ml:fr_core_news_sm'], 'prob': 1}, 'fields': {}, 'normalizations': {}}, {'classId': 'e_1', 'part': 's1v1', 'offsets': [{'start': 2963, 'text': 'Georges Duroy'}], 'coordinates': [], 'confidence': {'state': 'pre-added', 'who': ['m

In [20]:
def get_class_id_inv(label):
  """
  Translates the tagtog entity type id into spaCy label id 
  - label: tagtog entity type id
  """
  choices = {'e_1': 'PER', 'e_2': 'LOC', 'e_3': 'MISC'}
  return choices.get(label, None)

In [21]:
training_data = {'classes' : ['PER', "LOC", "MISC"], 'annotations' : []}
temp_dict = {}
temp_dict['text'] = text
temp_dict['entities'] = []
for example in js_corrected_training_data['entities']:  
  #print(example)
  start = example['offsets'][0]['start']
  end = int(example['offsets'][0]['start']) + len(example['offsets'][0]['text'])
  label = get_class_id_inv(example['classId'])
  temp_dict['entities'].append((start, end, label))
  #print(temp_dict['entities'])
training_data['annotations'].append(temp_dict)
  
print(training_data['annotations'][0])

{'text': "['PREMIÈRE PARTIE.', 'I', 'Quand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant.', 'Comme il portait beau, par nature et par pose d'ancien sous-officier, il cambra sa taille, frisa sa moustache d'un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s'étendent comme des coups d'épervier.', 'Les femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d'un chapeau toujours poussiéreux et vêtue d'une robe toujours de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.', 'Lorsqu'il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu'il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans dîner

**5.  Initialisation du modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement** 

In [22]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("fr") # load a new spacy model
doc_bin = DocBin() # create a DocBin object


In [23]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("coursTAL/2022/training_data.spacy") # save the docbin object


100%|██████████| 1/1 [00:00<00:00,  9.09it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


Le fichier de configuration base_config_nermodel.cfg est base_config_nermodel.cfg 

**6.  Initialisation de l'entrainement avec les paramètres définis (hyperparamètres) dans le fichier .cfg et entrainement par le biais du CLI de Spacy**

In [24]:
!python -m spacy init fill-config "coursTAL/2022/base_config_nermodel.cfg" "coursTAL/2022/config.cfg"

2022-12-07 04:13:37.320109: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
coursTAL/2022/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [25]:
!python -m spacy train "coursTAL/2022/config.cfg" --paths.train "coursTAL/2022/training_data.spacy" --paths.dev "coursTAL/2022/training_data.spacy" --output "coursTAL/2022/"


2022-12-07 04:13:56.385790: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: coursTAL/2022
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-07 04:14:03,229] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-07 04:14:03,246] [INFO] Pipeline: ['tok2vec', 'ner', 'parser']
INFO:spacy:Pipeline: ['tok2vec', 'ner', 'parser']
[2022-12-07 04:14:03,246] [INFO] Resuming training for: ['ner']
INFO:spacy:Resuming training for: ['ner']
[2022-12-07 04:14:03,258] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-07 04:14:03,259] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-07 04:14:04,562] [INFO] Initialized pipeline components: ['tok2vec', 'parser']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'parser']
✔ Initialized pipeline

=

**7. Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant**

In [34]:
nlp_ner = spacy.load("coursTAL/2022/model-best-NER-Eltec/model-best")

test_dataset = open("coursTAL/2022/belAmi_01-05.txt").read().replace("’","'")
doc = nlp_ner(test_dataset)

colors = {"PER": "#F67DE3", "LOC": "#7DF6D9", "MISC" : "#FFFFFF"}
options = {"colors": colors} 

show_ents(doc)

#spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Duroy - 85 - 90 - PER - Named person or family.


Duroy - 660 - 665 - PER - Named person or family.


Algérie - 851 - 858 - LOC - Non-GPE locations, mountain ranges, bodies of water


Bois - 1096 - 1100 - LOC - Non-GPE locations, mountain ranges, bodies of water


Mme Forestier - 1730 - 1743 - PER - Named person or family.


Mme - 1899 - 1902 - ROLE - None


de Marelle - 1903 - 1913 - PER - Named person or family.


rue de Verneuil - 2160 - 2175 - LOC - Non-GPE locations, mountain ranges, bodies of water


Duroy - 2425 - 2430 - PER - Named person or family.


Duroy - 3037 - 3042 - PER - Named person or family.


Mme de Marelle - 3115 - 3129 - PER - Named person or family.


Duroy - 3465 - 3470 - PER - Named person or family.


Norbert de Varenne - 3589 - 3607 - PER - Named person or family.


Paris - 3733 - 3738 - LOC - Non-GPE locations, mountain ranges, bodies of water


Mme Forestier - 4599 - 4612 - PER - Named person or family.


Auprès de Mme de Marelle - 5008 - 5032 - PER - Named

/usr/local/lib/python3.8/dist-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'ROLE' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/usr/local/lib/python3.8/dist-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'OTHER' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/usr/local/lib/python3.8/dist-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'WORK' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))


**A titre de comparaison, test avec ce même texte mais à partir du modèle de base fourni par Spacy, fr_core_news_sm.**

In [35]:
pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(test_dataset)
show_ents(doc)

V



Deux - 0 - 9 - PER - Named person or family.


Duroy - 85 - 90 - LOC - Non-GPE locations, mountain ranges, bodies of water


Duroy - 660 - 665 - LOC - Non-GPE locations, mountain ranges, bodies of water


Algérie - 851 - 858 - LOC - Non-GPE locations, mountain ranges, bodies of water


Bois - 1096 - 1100 - LOC - Non-GPE locations, mountain ranges, bodies of water


Mme Forestier - 1730 - 1743 - PER - Named person or family.


Mme de Marelle - 1899 - 1913 - PER - Named person or family.


rue de Verneuil - 2160 - 2175 - LOC - Non-GPE locations, mountain ranges, bodies of water


madame - 2310 - 2316 - PER - Named person or family.


Duroy - 2425 - 2430 - LOC - Non-GPE locations, mountain ranges, bodies of water


Duroy - 3037 - 3042 - LOC - Non-GPE locations, mountain ranges, bodies of water


Mme de Marelle - 3115 - 3129 - PER - Named person or family.


Duroy - 3465 - 3470 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Norbert de Varenne -

Remarque 1. lors de la mise à jour du modèle NER (idem pour d'autres couches comme POS) avec les annotations de l'utilisateur, un processus itératif se met en place afin de faire converger les prédictions faites (des poids) vers des valeurs optimales vis-à-vis des annotations de référence. A caque itération, on calcule le "gradient" qui correspond, en optimisation, aux dérivées partielles de la fonction de classification non linéaire recherchée (voir [1], vous trouverez aussi la notion de backpropagation clé dans ces approches de DL pour le TAL). Pour observer la manière dont la valeur du gradient évolue (et se stabilise) lors des itérations, il est conseillé de regarder le paramètre "losses" mis à jour par la méthode update(..., losses=losses).
[1]: https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1162/handouts/CS224N_DeepNLP_Week7_lecture3.pdf 
Aussi : https://spacy.io/usage/training

Remarque 2. Pour éviter le surapprentissage (overfitting) d'un modèle : vous pouvez identifier que votre modèle n'est pas bon lorsqu'il fonctionne bien sur les données d'entraînement mais ne donne pas de bons résultats sur des données nouvelles et non encore vues. Autrement dit, le modèle "mémorise" les données d'apprentissage et n'est pas performant avec les nouvelles données.